In [1]:
#!pip install h2o
import pandas as pd
import h2o
from h2o.automl import H2OAutoML
from benchmark_utils import timer


In [2]:
h2o.init()
X_train = h2o.import_file('../datasets/house-prices/train.csv',header=1)
X_test = h2o.import_file('../datasets/house-prices/test.csv',header=1)

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,44 secs
H2O cluster timezone:,America/Sao_Paulo
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.5
H2O cluster version age:,1 month and 25 days
H2O cluster name:,H2O_from_python_midas_dni6gj
H2O cluster total nodes:,1
H2O cluster free memory:,22.37 Gb
H2O cluster total cores:,20
H2O cluster allowed cores:,20
H2O cluster status:,"locked, healthy"


Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [3]:
X_train.head()

Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
1,60,RL,65,8450,Pave,NA,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NA,Attchd,2003,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NA,NA,NA,0,2,2008,WD,Normal,208500
2,20,RL,80,9600,Pave,NA,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NA,NA,NA,0,5,2007,WD,Normal,181500
3,60,RL,68,11250,Pave,NA,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NA,NA,NA,0,9,2008,WD,Normal,223500
4,70,RL,60,9550,Pave,NA,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NA,NA,NA,0,2,2006,WD,Abnorml,140000
5,60,RL,84,14260,Pave,NA,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NA,NA,NA,0,12,2008,WD,Normal,250000
6,50,RL,85,14115,Pave,NA,IR1,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,1.5Fin,5,5,1993,1995,Gable,CompShg,VinylSd,VinylSd,None,0,TA,TA,Wood,Gd,TA,No,GLQ,732,Unf,0,64,796,GasA,Ex,Y,SBrkr,796,566,0,1362,1,0,1,1,1,1,TA,5,Typ,0,NA,Attchd,1993,Unf,2,480,TA,TA,Y,40,30,0,320,0,0,NA,MnPrv,Shed,700,10,2009,WD,Normal,143000
7,20,RL,75,10084,Pave,NA,Reg,Lvl,AllPub,Inside,Gtl,Somerst,Norm,Norm,1Fam,1Story,8,5,2004,2005,Gable,CompShg,VinylSd,VinylSd,Stone,186,Gd,TA,PConc,Ex,TA,Av,GLQ,1369,Unf,0,317,1686,GasA,Ex,Y,SBrkr,1694,0,0,1694,1,0,2,0,3,1,Gd,7,Typ,1,Gd,Attchd,2004,RFn,2,636,TA,TA,Y,255,57,0,0,0,0,NA,NA,NA,0,8,2007,WD,Normal,307000
8,60,RL,nan,10382,Pave,NA,IR1,Lvl,AllPub,Corner,Gtl,NWAmes,PosN,Norm,1Fam,2Story,7,6,1973,1973,Gable,CompShg,HdBoard,HdBoard,Stone,240,TA,TA,CBlock,Gd,TA,Mn,ALQ,859,BLQ,32,216,1107,GasA,Ex,Y,SBrkr,1107,983,0,2090,1,0,2,1,3,1,TA,7,Typ,2,TA,Attchd,1973,RFn,2,484,TA,TA,Y,235,204,228,0,0,0,NA,NA,Shed,350,11,2009,WD,Normal,200000
9,50,RM,51,6120,Pave,NA,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Artery,Norm,1Fam,1.5Fin,7,5,1931,1950,Gable,CompShg,BrkFace,Wd Shng,None,0,TA,TA,BrkTil,TA,TA,No,Unf,0,Unf,0,952,952,GasA,Gd,Y,FuseF,1022,752,0,1774,0,0,2,0,2,2,TA,8,Min1,2,TA,Detchd,1931,Unf,2,468,Fa,TA,Y,90,0,205,0,0,0,NA,NA,NA,0,4,2008,WD,Abnorml,129900
10,190,RL,50,7420,Pave,NA,Reg,Lvl,AllPub,Corner,Gtl,BrkSide,Artery,Artery,2fmCon,1.5Unf,5,6,1939,1950,Gable,CompShg,MetalSd,MetalSd,None,0,TA,TA,BrkTil,TA,TA,No,GLQ,851,Unf,0,140,991,GasA,Ex,Y,SBrkr,1077,0,0,1077,1,0,1,0,2,2,TA,5,Typ,2,TA,Attchd,1939,RFn,1,205,Gd,TA,Y,0,4,0,0,0,0,NA,NA,NA,0,1,2008,

## Treinamento

In [4]:
start_time = timer(None)

x = X_train.columns
y = "SalePrice"


# (limited to 1 hour max runtime by default)
aml = H2OAutoML()
aml.train(x=x, y=y, training_frame=X_train)
time = timer(start_time)



AutoML progress: |████████████████████████████████████████████████████████| 100%


In [5]:
print(time)



 Time taken: 0 hours 49 minutes and 50.29 seconds.


In [6]:
# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)


model_id,mean_residual_deviance,rmse,mse,mae,rmsle
GBM_grid_1_AutoML_20191111_121248_model_4,7.47105e+08,27333.2,7.47105e+08,15617.9,0.130777
GBM_1_AutoML_20191111_121248,7.57227e+08,27517.8,7.57227e+08,15761.5,0.133428
GBM_3_AutoML_20191111_121248,7.95469e+08,28204.1,7.95469e+08,15921.5,0.134276
GBM_grid_1_AutoML_20191111_121248_model_13,7.98243e+08,28253.2,7.98243e+08,16070.5,0.133676
GBM_4_AutoML_20191111_121248,7.98432e+08,28256.5,7.98432e+08,16211.5,0.135003
GBM_grid_1_AutoML_20191111_121248_model_2,8.10743e+08,28473.6,8.10743e+08,16317.5,0.134372
GBM_2_AutoML_20191111_121248,8.2388e+08,28703.3,8.2388e+08,16241.8,0.13472
DRF_1_AutoML_20191111_121248,8.33527e+08,28870.9,8.33527e+08,17010.8,0.143238
GBM_grid_1_AutoML_20191111_121248_model_15,8.43477e+08,29042.7,8.43477e+08,17107.7,0.145026
XRT_1_AutoML_20191111_121248,8.65292e+08,29415.8,8.65292e+08,16964.6,0.14392


In [7]:
# The leader model is stored here
aml.leader

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_grid_1_AutoML_20191111_121248_model_4


Model Summary: 

,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,84.0,84.0,40116.0,7.0,16.0,12.595238,28.0,37.0,33.02381




ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 313703176.9972049
RMSE: 17711.667820880248
MAE: 8769.739891909247
RMSLE: 0.07837929526593647
Mean Residual Deviance: 313703176.9972049

ModelMetricsRegression: gbm
** Reported on cross-validation data. **

MSE: 747105074.5715325
RMSE: 27333.222908605792
MAE: 15617.870512697498
RMSLE: 0.13077744211794678
Mean Residual Deviance: 747105074.5715325

Cross-Validation Metrics Summary: 

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,mae,15617.87,2145.6035,14005.665,14812.178,15014.756,19391.02,14865.735
1,mean_residual_deviance,7.4710509E8,3.74447872E8,3.99881344E8,6.6743482E8,6.660425E8,1.38731648E9,6.1485024E8
2,mse,7.4710509E8,3.74447872E8,3.99881344E8,6.6743482E8,6.660425E8,1.38731648E9,6.1485024E8
3,r2,0.8857853,0.036916986,0.9256277,0.8946659,0.8905205,0.8248958,0.89321643
4,residual_deviance,7.4710509E8,3.74447872E8,3.99881344E8,6.6743482E8,6.660425E8,1.38731648E9,6.1485024E8
5,rmse,26736.492,6350.6924,19997.033,25834.76,25807.799,37246.7,24796.174
6,rmsle,0.12974772,0.018312123,0.13347968,0.113557145,0.1305579,0.15789653,0.11324738



Scoring History: 

,,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance
0,,2019-11-11 12:23:55,4 min 17.247 sec,0.0,79415.291886,57434.770276,6.306789e+09
1,,2019-11-11 12:23:55,4 min 17.764 sec,5.0,59377.483478,41225.220088,3.525686e+09
2,,2019-11-11 12:23:55,4 min 17.901 sec,10.0,46369.675760,30509.806849,2.150147e+09
3,,2019-11-11 12:23:56,4 min 18.236 sec,15.0,37522.040812,23122.638490,1.407904e+09
4,,2019-11-11 12:23:56,4 min 18.942 sec,20.0,31372.236256,18315.358888,9.842172e+08
5,,2019-11-11 12:23:57,4 min 19.888 sec,25.0,27863.763454,15639.007069,7.763893e+08
6,,2019-11-11 12:23:58,4 min 20.601 sec,30.0,25483.466897,13842.914148,6.494071e+08
7,,2019-11-11 12:23:58,4 min 21.099 sec,35.0,23967.436272,12772.649299,5.744380e+08
8,,2019-11-11 12:23:59,4 min 21.808 sec,40.0,22771.037202,11929.848352,5.185201e+08
9,,2019-11-11 12:24:00,4 min 22.720 sec,45.0,21877.016160,11347.967113,4.786038e+08



Variable Importances: 

,variable,relative_importance,scaled_importance,percentage
0,OverallQual,1.670454e+13,1.000000,0.307024
1,Neighborhood,4.568013e+12,0.273459,0.083959
2,GarageCars,4.414023e+12,0.264241,0.081128
3,GrLivArea,3.553269e+12,0.212713,0.065308
4,1stFlrSF,3.472066e+12,0.207852,0.063815
5,YearBuilt,3.287146e+12,0.196782,0.060417
6,GarageArea,2.567457e+12,0.153698,0.047189
7,KitchenQual,2.289067e+12,0.137033,0.042072
8,ExterQual,2.157170e+12,0.129137,0.039648
9,BsmtFinSF1,1.494131e+12,0.089445,0.027462



See the whole table with table.as_data_frame()


In [8]:
preds = aml.leader.predict(X_test)
print(preds)




gbm prediction progress: |████████████████████████████████████████████████| 100%


/home/midas/anaconda3/envs/automl-3.7/lib/python3.7/site-packages/h2o/job.py:70: UserWarning: Test/Validation dataset column 'MSZoning' has levels not trained on: [NA]
  warnings.warn(w)
/home/midas/anaconda3/envs/automl-3.7/lib/python3.7/site-packages/h2o/job.py:70: UserWarning: Test/Validation dataset column 'Utilities' has levels not trained on: [NA]
  warnings.warn(w)
/home/midas/anaconda3/envs/automl-3.7/lib/python3.7/site-packages/h2o/job.py:70: UserWarning: Test/Validation dataset column 'Exterior1st' has levels not trained on: [NA]
  warnings.warn(w)
/home/midas/anaconda3/envs/automl-3.7/lib/python3.7/site-packages/h2o/job.py:70: UserWarning: Test/Validation dataset column 'Exterior2nd' has levels not trained on: [NA]
  warnings.warn(w)
/home/midas/anaconda3/envs/automl-3.7/lib/python3.7/site-packages/h2o/job.py:70: UserWarning: Test/Validation dataset column 'KitchenQual' has levels not trained on: [NA]
  warnings.warn(w)
/home/midas/anaconda3/envs/automl-3.7/lib/python3.7/sit

predict
126334
163454
181981
193408
181456
181550
173764
171513
189081
130146


In [9]:
#h2o.save_model(aml.leader, path = "House_prices_h2o")

In [10]:
print(time)

id_test = X_test.as_data_frame()["Id"]
preds_final = preds.as_data_frame()["predict"]

print(preds)


 Time taken: 0 hours 49 minutes and 50.29 seconds.


predict
126334
163454
181981
193408
181456
181550
173764
171513
189081
130146


In [11]:
submission = pd.DataFrame({
        "id": id_test,
        "target": preds_final
    })

submission.to_csv("../submit_files/house_prices_h2o_submission.csv", index=False)